In [137]:
import requests
from bs4 import BeautifulSoup as Soup
import json
import pandas as pd

from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults
from pyzillow.pyzillow import ZillowWrapper, GetUpdatedPropertyDetails

ZWSID = ''
#zillow_data = ZillowWrapper(ZWSID)

In [138]:
# query https://www.zillow.com/homes/recently_sold/house_type/90_days/pricea_sort/45.601969,-122.744751,45.504362,-122.894611_rect/12_zm/3_p/
# zipcode 97229, all houses sold in last 90 days
query_page = "https://www.zillow.com/search/GetResults.htm?spt=homes&status=001000&lt=000000&ht=100000&pr=,&mp=,&bd=0%2C&ba=0%2C&sf=,&lot=0%2C&yr=,&singlestory=0&hoa=0%2C&pho=0&pets=0&parking=0&laundry=0&income-restricted=0&pnd=0&red=0&zso=0&days=6m&ds=all&pmf=0&pf=0&sch=100111&zoom=12&rect=-122912035,45533469,-122727499,45572897&sort=pricea&search=maplist&listright=true&isMapSearch=true&zoom=12&p=1"

In [139]:
print query_page

https://www.zillow.com/search/GetResults.htm?spt=homes&status=001000&lt=000000&ht=100000&pr=,&mp=,&bd=0%2C&ba=0%2C&sf=,&lot=0%2C&yr=,&singlestory=0&hoa=0%2C&pho=0&pets=0&parking=0&laundry=0&income-restricted=0&pnd=0&red=0&zso=0&days=6m&ds=all&pmf=0&pf=0&sch=100111&zoom=12&rect=-122912035,45533469,-122727499,45572897&sort=pricea&search=maplist&listright=true&isMapSearch=true&zoom=12&p=1


In [140]:
response = requests.get(query_page) # request the json file
json_response =  json.loads(response.text) # parse the json file
num_pages = json_response['list']['numPages']

In [141]:
base_page = "https://www.zillow.com/search/GetResults.htm?spt=homes&status=001000&lt=000000&ht=100000&pr=,&mp=,&bd=0%2C&ba=0%2C&sf=,&lot=0%2C&yr=,&singlestory=0&hoa=0%2C&pho=0&pets=0&parking=0&laundry=0&income-restricted=0&pnd=0&red=0&zso=0&days=6m&ds=all&pmf=0&pf=0&sch=100111&zoom=12&rect=-122912035,45533469,-122727499,45572897&sort=pricea&search=maplist&listright=true&isMapSearch=true&zoom=12"

In [155]:
rows_list = []

for page in range(1,2):
    url = base_page + "&p=" + str(page)
    response = requests.get(url) # request the json file
    json_response =  json.loads(response.text) # parse the json file
    soup = Soup(json_response['list']['listHTML'], 'html.parser')
    links = [i.attrs['href'] for i in soup.findAll("a",{"class":"hdp-link"})]
    for link in range(len(links)):
        
        address = ''
        for txt in links[link].split('/')[2].split('-')[:-1]:
            address += txt + ' '
        zipcode = links[link].split('/')[2].split('-')[-1]
        address = address.rstrip()
        
        zillow_data = ZillowWrapper(ZWSID)
        print address, zipcode
        deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
        
        if (zillow_data):
            deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
            if (deep_search_response):
                result = GetDeepSearchResults(deep_search_response)
        
                if (result):
                    zillow_id = result.zillow_id
                    print zillow_id, address, zipcode

                    home_dict = {}
                    home_dict['zillow_id'] = result.zillow_id
                    home_dict['address'] = address
                    home_dict['zipcode'] = zipcode
                    home_dict['home_type'] = result.home_type
                    home_dict['home_detail_link'] = result.home_detail_link
                    home_dict['graph_data_link'] = result.graph_data_link
                    home_dict['map_this_home_link'] = result.map_this_home_link
                    home_dict['latitude'] = result.latitude
                    home_dict['longitude'] = result.longitude
                #home_dict['coordinates'] = result.coordinates
                    home_dict['tax_year'] = result.tax_year
                    home_dict['tax_value'] = result.tax_value
                    home_dict['year_built'] = result.year_built
                    home_dict['property_size'] = result.property_size
                    home_dict['home_size'] = result.home_size
                    home_dict['bathrooms'] = result.bathrooms
                    home_dict['bedrooms'] = result.bedrooms
                    home_dict['last_sold_date'] = result.last_sold_date
                    home_dict['last_sold_price_currency'] = result.last_sold_price_currency
                    home_dict['last_sold_price'] = result.last_sold_price
        
        rows_list.append(home_dict)
    
df = pd.DataFrame(rows_list)    
    

19735 NW Quail Hollow Dr Portland OR 97229
48617639 19735 NW Quail Hollow Dr Portland OR 97229
17477 NW Carl Ct Portland OR 97229
48539864 17477 NW Carl Ct Portland OR 97229
16683 NW Viola St Portland OR 97229
126290884 16683 NW Viola St Portland OR 97229
15130 NW Oakhills Dr Beaverton OR 97006
48612790 15130 NW Oakhills Dr Beaverton OR 97006
2225 NW Wheatfield Way Portland OR 97229
48563932 2225 NW Wheatfield Way Portland OR 97229
17516 NW Springville Rd APT B5 Portland OR 97229
55528064 17516 NW Springville Rd APT B5 Portland OR 97229
15325 NW Central Dr UNIT 313 Portland OR 97229
67926194 15325 NW Central Dr UNIT 313 Portland OR 97229
19771 NW Quail Hollow Dr Portland OR 97229
48617649 19771 NW Quail Hollow Dr Portland OR 97229
3755 NW Olympic Dr Portland OR 97229
48613466 3755 NW Olympic Dr Portland OR 97229
5425 NW Ridgemoor Ct Portland OR 97229
48617689 5425 NW Ridgemoor Ct Portland OR 97229
6637 NW Saint Helens Rd Portland OR 97210
53987585 6637 NW Saint Helens Rd Portland OR 97

In [152]:
rows_list = []

for page in range(1,num_pages):
    url = base_page + "&p=" + str(page)
    response = requests.get(url) # request the json file
    json_response =  json.loads(response.text) # parse the json file
    soup = Soup(json_response['list']['listHTML'], 'html.parser')
    links = [i.attrs['href'] for i in soup.findAll("a",{"class":"hdp-link"})]
    for link in range(len(links)):
        
        address = ''
        for txt in links[link].split('/')[2].split('-')[:-1]:
            address += txt + ' '
        zipcode = links[link].split('/')[2].split('-')[-1]
        address = address.rstrip()
        
        zillow_data = ZillowWrapper(ZWSID)
        deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
        
        if (zillow_data):
            deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
            if (deep_search_response):
                result = GetDeepSearchResults(deep_search_response)
        
                if (result):
                    zillow_id = result.zillow_id
                    print zillow_id, address, zipcode

                    home_dict = {}
                    home_dict['zillow_id'] = result.zillow_id
                    home_dict['address'] = address
                    home_dict['zipcode'] = zipcode
                    home_dict['home_type'] = result.home_type
                    home_dict['home_detail_link'] = result.home_detail_link
                    home_dict['graph_data_link'] = result.graph_data_link
                    home_dict['map_this_home_link'] = result.map_this_home_link
                    home_dict['latitude'] = result.latitude
                    home_dict['longitude'] = result.longitude
                #home_dict['coordinates'] = result.coordinates
                    home_dict['tax_year'] = result.tax_year
                    home_dict['tax_value'] = result.tax_value
                    home_dict['year_built'] = result.year_built
                    home_dict['property_size'] = result.property_size
                    home_dict['home_size'] = result.home_size
                    home_dict['bathrooms'] = result.bathrooms
                    home_dict['bedrooms'] = result.bedrooms
                    home_dict['last_sold_date'] = result.last_sold_date
                    home_dict['last_sold_price_currency'] = result.last_sold_price_currency
                    home_dict['last_sold_price'] = result.last_sold_price

                    updated_property_details_response = zillow_data.get_updated_property_details(zillow_id)
                    if (updated_property_details_response):
                        result2 = GetUpdatedPropertyDetails(updated_property_details_response)
                
#            home_dict['zillow_id2'] = result2.zillow_id
#            home_dict['home_type2'] = result2.home_type
#            home_dict['home_detail_link2'] = result2.home_detail_link
#            home_dict['photo_gallery'] = result2.photo_gallery
#            home_dict['latitude2'] = result2.latitude
#            home_dict['longitude2'] = result2.longitude
#            home_dict['coordinates'] = result2.coordinates
#            home_dict['year_built2'] = result2.year_built
#            home_dict['property_size2'] = result2.property_size
#            home_dict['home_size2'] = result2.home_size
#            home_dict['bathrooms2'] = result2.bathrooms
#            home_dict['bedrooms2'] = result2.bedrooms
#            home_dict['home_info'] = result2.home_info

                        if (result2):
                            print "I am here"
                            home_dict['year_updated'] = result2.year_updated
                            home_dict['floors'] = result2.floors
                            home_dict['basement'] = result2.basement
                            home_dict['roof'] = result2.roof
                            home_dict['view'] = result2.view
                            home_dict['heating_sources'] = result2.heating_sources
                            home_dict['heating_system'] = result2.heating_system
                            home_dict['rooms'] = result2.rooms
                            home_dict['neighborhood'] = result2.neighborhood
                            home_dict['school_district'] = result2.school_district
            
            
                    rows_list.append(home_dict)
    
df = pd.DataFrame(rows_list)

48617639 19735 NW Quail Hollow Dr Portland OR 97229


TypeError: coercing to Unicode: need string or buffer, dict found

In [156]:
print zillow_id

48552671


In [157]:
updated_property_details_response = zillow_data.get_updated_property_details(zillow_id)

In [158]:
if (updated_property_details_response):
    result2 = GetUpdatedPropertyDetails(updated_property_details_response)

In [159]:
print result2

48552671


In [163]:
home_dict['year_updated'] = result2.year_updated
#home_dict['floors'] = result2.floors
home_dict['basement'] = result2.basement
home_dict['roof'] = result2.roof
home_dict['view'] = result2.view
home_dict['heating_sources'] = result2.heating_sources
home_dict['heating_system'] = result2.heating_system
home_dict['rooms'] = result2.rooms
home_dict['neighborhood'] = result2.neighborhood
home_dict['school_district'] = result2.school_district

In [164]:
home_dict

{'address': u'17936 NW Deerfield Dr Portland OR',
 'basement': None,
 'bathrooms': '3.0',
 'bedrooms': '3',
 'graph_data_link': 'http://www.zillow.com/homedetails/17936-NW-Deerfield-Dr-Portland-OR-97229/48552671_zpid/#charts-and-data',
 'heating_sources': None,
 'heating_system': None,
 'home_detail_link': 'http://www.zillow.com/homedetails/17936-NW-Deerfield-Dr-Portland-OR-97229/48552671_zpid/',
 'home_size': '1690',
 'home_type': 'SingleFamily',
 'last_sold_date': '12/03/2016',
 'last_sold_price': '332500',
 'last_sold_price_currency': 'USD',
 'latitude': '45.558525',
 'longitude': '-122.861822',
 'map_this_home_link': 'http://www.zillow.com/homes/48552671_zpid/',
 'neighborhood': None,
 'property_size': '4792',
 'roof': None,
 'rooms': None,
 'school_district': None,
 'tax_value': '302040.0',
 'tax_year': '2015',
 'view': None,
 'year_built': '1990',
 'year_updated': None,
 'zillow_id': '48552671',
 'zipcode': u'97229'}

In [143]:
df.count()

address                     352
bathrooms                   352
bedrooms                    352
graph_data_link             352
home_detail_link            352
home_size                   352
home_type                   352
last_sold_date              352
last_sold_price             352
last_sold_price_currency    352
latitude                    352
longitude                   352
map_this_home_link          352
property_size               352
tax_value                   352
tax_year                    352
year_built                  352
zillow_id                   352
zipcode                     352
dtype: int64

In [144]:
df.head(52)

,address,bathrooms,bedrooms,graph_data_link,home_detail_link,home_size,home_type,last_sold_date,last_sold_price,last_sold_price_currency,latitude,longitude,map_this_home_link,property_size,tax_value,tax_year,year_built,zillow_id,zipcode
0,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
1,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
2,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
3,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
4,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
5,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
6,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
7,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
8,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
9,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229


In [145]:
df.describe()

,address,bathrooms,bedrooms,graph_data_link,home_detail_link,home_size,home_type,last_sold_date,last_sold_price,last_sold_price_currency,latitude,longitude,map_this_home_link,property_size,tax_value,tax_year,year_built,zillow_id,zipcode
count,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,19735 NW Quail Hollow Dr Portland OR,2.0,2,http://www.zillow.com/homedetails/19735-NW-Qua...,http://www.zillow.com/homedetails/19735-NW-Qua...,958,SingleFamily,12/29/2016,5000,USD,45.559108,-122.879846,http://www.zillow.com/homes/48617639_zpid/,1618,150150.0,2015,1978,48617639,97229
freq,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352


In [5]:
json_response

{u'breadcrumb': {},
 u'filters': {u'summary': u'<div id="sf11-title" class="title">Other Filters</div><div id="applied-summary">Sold, Home Type: sfh, 6 months</div>'},
 u'list': {u'binCounts': {u'bankOwnedCount': 0,
   u'comingSoonCount': 0,
   u'condoCount': 0,
   u'forRentCount': 4,
   u'forSaleCount': 0,
   u'foreclosedCount': 0,
   u'foreclosuresCount': 0,
   u'fsbaCount': 4,
   u'fsboCount': 0,
   u'landCount': 0,
   u'makeMeMoveCount': 0,
   u'manufacturedCount': 0,
   u'multiFamilyCount': 0,
   u'newConstructionCount': 2,
   u'openHousesCount': 0,
   u'preForeclosureCount': 0,
   u'recentlySoldCount': 379,
   u'singleFamilyCount': 379,
   u'totalResultCount': 379,
   u'unmappedCount': 0},
  u'hasFavoriteClaimedProperty': False,
  u'listHTML': u'<div id="search-results" class="pop-bubble "><div id="home-owner-lightbox_src" class="template hide"><!--<div id="home\\-owner\\-lightbox_content" class="dialog clearfix"><a class="close lightbox\\-close zsg\\-icon\\-x\\-thin" title="Clos

In [146]:
df.to_csv("houses_97229.csv")

In [3]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

In [7]:
param_test1['min_child_weight']

[1, 3, 5]